In [2]:
from fastai2.basics           import *
from fastai2.vision.all       import *
from fastai2.medical.imaging  import *

np.set_printoptions(linewidth=120)
pd.options.display.max_rows = 999
matplotlib.rcParams['image.cmap'] = 'bone'
set_num_threads(1)
set_seed(42)

path = Path('~/data/rsna').expanduser()
path_trn = path/'stage_2_train'
path_tst = path/'stage_2_test'
path_jpg = path/'nocrop_jpg'
path_jpg256 = path/'nocrop_jpg256'

torch.cuda.set_device(1)

In [3]:
path_meta = path/'meta'
df_lbls = pd.read_feather(path_meta/'labels.fth')
df_tst = pd.read_feather(path_meta/'df_tst.fth')
df_trn = pd.read_feather(path_meta/'df_trn.fth').dropna(subset=['img_pct_window'])
bins = (path_meta/'bins.pkl').load()

In [4]:
# comb = df_trn.join(df_lbls.set_index('ID'), 'SOPInstanceUID')
# comb.reset_index(drop=True).to_feather(path_meta/'df_comb.fth')

In [5]:
df_comb = pd.read_feather(path_meta/'df_comb.fth')

In [6]:
qpath = Path('~/data/qure').expanduser()
qpath_meta = qpath/'meta'
qdf_comb = pd.read_feather(qpath_meta/'df_comb.fth')

In [7]:
sop = qdf_comb.SOPInstanceUID[0]

In [8]:
sop = df_comb.SOPInstanceUID[0]

In [9]:
def remove_suff(x): return '.'.join(x.split('.')[:-1])
def fname(fname): return path_jpg256/f'{fname}.jpg'

In [10]:
fn = fname(sop)

In [11]:
qdf_comb['is_qure'] = 1
df_comb['is_qure'] = 0

In [12]:
both_df_comb = pd.concat([df_comb, qdf_comb], sort=False, join='inner')

In [13]:
both_df_comb.reset_index(drop=True).to_feather(path_meta/'both_df_comb.fth')

In [14]:
patients = both_df_comb.PatientID.unique()
patient_grps = Path('patient_grps.pkl').load()
prev_patients = np.concatenate(patient_grps)
len(prev_patients),len(patients)

(19428, 19428)

In [31]:
patset1 = set(patients)
patset2 = set(prev_patients)
new_pat = array(list(patset1-patset2))
patient_grps[4] = np.concatenate([new_pat,patient_grps[4]])
Path('patient_grps2.pkl').save(patient_grps)